# LangChain


## 1. install

In [17]:
# source venv/bin/activate
# !pip install uvicorn
# !pip install --upgrade pip
# !pip install fastapi
# !pip install -qU langchain-openai
# !pip install langchain-teddynote
# !pip install confluent-kafka
# !pip install -U langchain-community

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.8 MB/s eta 0:00:00a 0:00:01
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


## 2. Test 1 : LangChain 
### LLM 개발, 모니터링 및 테스트 플랫폼
- 예상치 못한 최종 결과
- 에이전트가 루핑되는 이유
- 체인이 예상보다 느린 이유
- 에이전트가 각 단계에서 사용한 토큰 수
- reference : https://python.langchain.com/v0.2/docs/tutorials/chatbot/, https://wikidocs.net/250954
- LangSmith : https://smith.langchain.com/

### OpenAI Models
- temperature : 0-2, 낮을 수록 결정론적
- max_tokens : gpt-4o-mini, 16,384 / gpt-4o, 4,096 


In [1]:
import os
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_openai import ChatOpenAI


# .env 파일에서 환경 변수 로드
load_dotenv()
langsmith_project = os.getenv("LANGCHAIN_PROJECT")

logging.langsmith(langsmith_project)

llm = ChatOpenAI(
    temperature=0.1,            # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",   # 모델명
)

# 질의내용
question = "외국인 노동자가 한국에서 일 할 수 있으려면 어떻게 해야해?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI_labor
[답변]: content='외국인 노동자가 한국에서 일하기 위해서는 다음과 같은 절차를 따라야 합니다:\n\n1. **비자 종류 확인**: 한국에서 일하기 위해서는 적절한 비자를 취득해야 합니다. 일반적으로 외국인 노동자는 E-9(비숙련 노동자), E-7(전문직), E-2(외국어 강사) 등의 비자를 신청할 수 있습니다. 자신의 직업과 상황에 맞는 비자 종류를 확인해야 합니다.\n\n2. **고용주 찾기**: 한국에서 일할 고용주를 찾아야 합니다. 고용주는 외국인 노동자를 고용하기 위해 필요한 절차를 따라야 하며, 외국인 고용 신고를 해야 합니다.\n\n3. **비자 신청**: 고용주와 계약을 체결한 후, 해당 비자를 신청합니다. 비자 신청은 한국 대사관이나 영사관을 통해 할 수 있으며, 필요한 서류를 준비해야 합니다. 일반적으로 필요한 서류는 다음과 같습니다:\n   - 비자 신청서\n   - 여권 사본\n   - 고용 계약서\n   - 고용주의 사업자 등록증\n   - 건강 진단서 등\n\n4. **비자 발급**: 비자 신청이 승인되면 비자를 발급받게 됩니다. 이후 한국에 입국할 수 있습니다.\n\n5. **체류 자격 변경 (필요 시)**: 한국에 입국한 후, 만약 다른 종류의 비자로 변경하고 싶다면, 출입국관리사무소에 가서 체류 자격 변경 신청을 해야 합니다.\n\n6. **근로계약 체결**: 한국에 도착한 후, 고용주와 근로계약을 체결하고, 근로 조건을 명확히 해야 합니다.\n\n7. **법적 절차 준수**: 한국에서 일하는 동안에는 한국의 노동법과 관련 법규를 준수해야 하며, 비자 조건을 지켜야 합니다.\n\n각 단계에서 필요한 서류나 절차는 개인의 상황에 따라 다를 수 있으므로, 구체적인 정보는 한국 출입국관리사무소나 대사관에 문의하는 것이 좋습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'co

## 3. Test 2 : LangChain + Faiss
- OpenAIEmbeddings : OpenAI API를 통해 텍스트 -> 벡터화
- Docstore : LangChain에서 문서(document) 저장소 역할. FAISS 벡터 저장소를 사용할 때 실제 텍스트와 메타데이터를 저장
- InMemoryDocstore : 메모리 내에서 문서를 저장, 관리하는 Docstore의 구현체. 모든 문서를 메모리에 저장해 빠른 조회가 가능.
- Document : 텍스트 데이터(Body)와 그에 대한 메타데이터(출처 - Law name, Chapter, Title)를 담고 있는 객체
- VectorStore : 벡터화된 데이터 저장소(vector DB) 임베딩 저장, 유사도 검색, 임베딩 관리, ex) FAISS
- Retriever : vector_store를 통해 만듦. 사용자의 질문과 관련된 문서를 검색, 질문을 임베딩으로 변환해 유사한 문서 검색
- ChatPromptTemplate : LLM에 전달할 입력을 정의 ( 검색한 문서 + 사용자 질문 )
- RetrieverQA : 검색과 답변 생성을 통합하는 체인. Retriever를 사용해 검색된 문서들을 기반으로 LLM이 답변을 생성하도록 연결

### 흐름
1) FAISS로 구성된 vector store를 LangChain의 저장소 형태인 Docstore 형태로 저장하는데, 이때 InMemoryDocstore를 사용한다.
2) 사용자의 질문이 들어오면, 그 질문을 OpenAIEmbeddings를 이용해서 벡터화 한다.
3) 벡터화한 질문을 retriever를 통해 유사한 문서를 검색한다.
4) RetrieverQA(질의응답 체인)를 통해 검색된 문서와 사용자의 질문을 LLM 모델로 전달한다.

In [1]:
import faiss
import os
import json
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document

# .env 파일에서 환경 변수 로드
load_dotenv()
project_api_key = os.getenv("OPENAI_API_KEY")
organization_id = os.getenv("ORGANIZATION_ID")
project_id = os.getenv("PROJECT_ID")
langsmith_project = os.getenv("LANGCHAIN_PROJECT")

# OpenAI LLM 설정 (GPT-4o-mini)
llm = ChatOpenAI(
    temperature=0.1,            # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",   # 모델명
)

# 벡터 스토어 설정
def setup_faiss_vectorstore():
    # FAISS 인덱스 로드
    index = faiss.read_index("law_embeddings.index")

    # JSON 파일에서 문서 로드
    with open("law_embeddings.json", "r", encoding="utf-8") as f:
        docs_data = json.load(f)

    # OpenAI 임베딩 모델 설정
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    # JSON 구조에 따라 Document 객체로 변환
    docs = []
    for item in docs_data:
        page_content = item.get('Body', '')  # 'Body' 필드 사용
        metadata = {
            "law": item.get('Law', ''),      # 'Law' 필드 사용
            "chapter": item.get('Chapter', ''),  # 'Chapter' 필드 사용
            "title": item.get('Title', '')   # 'Title' 필드 사용
        }
        doc = Document(page_content=page_content, metadata=metadata)
        docs.append(doc)

    # InMemoryDocstore에 문서 저장
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})

    # FAISS 벡터스토어 초기화
    vectorstore = FAISS(
        embedding_function=embeddings,      # 임베딩 생성기
        index=index,                        # 검색 인덱스
        docstore=docstore,                  # 문서 저장소    
        index_to_docstore_id={i: str(i) for i in range(len(docs))}  # 인덱스와 문서 ID 매핑
    )
    return vectorstore

# 질의응답 체인 설정
def setup_rag_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    system_prompt = (
        "You are an AI chatbot specialized in Korean labor law. "
        "Use the following pieces of retrieved context to answer the question. "
        "If you don't know the answer, say that you don't know. "
        "Please explain in as much detail as possible."
        "Always explain with numbers"
        "Please mention exactly which 'law' and 'title' you referenced in the retrieved context."
        "If possible, please also provide the number or link of the relevant department."
        "If it seems like you need to consult with a professional labor attorney, "
        "Please explain it to me as easily as possible, like a friend."
        # "please add the comment at the end, '전문 노무사와 매칭 서비스를 신청하시겠습니까?'"
        "\n\n"
        "{context}"
    )

    # 시스템 프롬프트 정의

    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    # 문서 조각을 결합하는 체인 생성
    question_answer_chain = create_stuff_documents_chain(llm, prompt)

    # 검색과 QA를 결합하는 체인 생성
    chain = create_retrieval_chain(retriever, question_answer_chain)

    return chain

# 질문 처리 및 답변 생성
def ask_labor_law_question(rag_chain, question):
    result = rag_chain.invoke({"input": question})  # 'input'을 사용하여 invoke 호출
    answer = result.get('answer', '답변을 찾을 수 없습니다.')  # 'answer' 키로 답변 가져오기
    source_docs = result.get('context', [])  # 참조된 문서 가져오기
    return answer, source_docs

# 벡터스토어 설정 (한 번만 설정)
vectorstore = setup_faiss_vectorstore()

# 질의응답 체인 설정 (한 번만 설정)
rag_chain = setup_rag_chain(vectorstore)


In [ ]:

# 메인 실행 함수
if __name__ == "__main__":
    # 벡터스토어 설정 (한 번만 설정)
    vectorstore = setup_faiss_vectorstore()

    # 질의응답 체인 설정 (한 번만 설정)
    rag_chain = setup_rag_chain(vectorstore)

    question = "안녕하세요. 저는 한국에서 일하고 있는 외국인 근로자입니다. 몇 개월 동안 임금을 받지 못했어요. 불법체류자인 제가 어떻게 해야하죠?"

    answer, source_docs = ask_labor_law_question(rag_chain, question)

    # 출력 형식에 맞게 답변과 참조 문서 출력
    print(f"[답변]: {answer}\n")
    print("[참고 문서]:")
    if source_docs:
        for idx, doc in enumerate(source_docs, 1):
            print(f"{idx}. {doc.metadata['law']}, {doc.metadata['chapter']}, {doc.metadata['title']}")
    else:
        print("참조된 문서가 없습니다.")


question = "안녕하세요. 저는 한국에서 일하고 있는 외국인 근로자입니다. 몇 개월 동안 임금을 받지 못했어요. 불법체류자인 제가 어떻게 해야하죠?"

## 4. Test 3 : Langchain + FAISS 성능 측정

### 질문 리스트

In [3]:
# 기본적인 법률 목록을 요구하는 질문 (5~10개)
questions = [
    "외국인 근로자를 보호하는 주요 노동법은 무엇인가요?",
    "한국에서 외국인 근로자에게 적용되는 기본적인 노동법 목록을 알려주세요.",
    "외국인 근로자의 고용 및 해고 절차를 다루는 법률에는 어떤 것들이 있나요?",
    "외국인 근로자에게 적용되는 한국의 최저임금 관련 법률 목록을 알고 싶어요.",
    "한국에서 외국인 근로자와 관련된 산업재해보상 관련 법률에는 어떤 것들이 있나요?",
    "외국인 근로자의 고용 안정과 관련된 주요 법률은 무엇인가요?",
    "한국에서 외국인 근로자를 위한 복지 및 혜택 관련 법률 목록을 알려주세요.",
    "불법 체류 중인 외국인 근로자도 보호받을 수 있는 노동법은 무엇인가요?",
    "외국인 근로자에게 적용되는 한국의 근로 시간 및 휴가 관련 법률 목록을 알려주세요.",
    "외국인 근로자를 위한 근로 환경 보호 법률은 무엇인가요?",

    # 구체적인 상황에서의 법률 정보를 요구하는 질문 (10~15개)
    "한국에서 임금을 체불당한 외국인 근로자가 보호받을 수 있는 법률은 무엇인가요?",
    "외국인 근로자가 산업재해로 인해 다쳤을 때 적용될 수 있는 법률 목록을 알고 싶습니다.",
    "외국인 근로자가 부당 해고를 당했을 때 구제받을 수 있는 법률에는 어떤 것들이 있나요?",
    "비자 문제로 해고당한 외국인 근로자를 보호하는 법률은 무엇인가요?",
    "외국인 근로자가 노동 계약서 없이 일하고 있을 때 보호받을 수 있는 법률은 무엇인가요?",
    "외국인 여성 근로자를 위한 성차별 방지 및 고용 평등 관련 법률 목록을 알려주세요.",
    "외국인 근로자가 야근이나 초과근무를 강요받았을 때 보호할 수 있는 법률은 무엇인가요?",
    "외국인 근로자가 고용 계약을 중도 해지했을 때 적용되는 법률 목록을 알고 싶어요.",
    "외국인 근로자가 최저임금 이하로 임금을 받았을 때 도움을 받을 수 있는 법률은 무엇인가요?",
    "외국인 근로자가 고용주로부터 폭언이나 괴롭힘을 당했을 때 구제할 수 있는 법률은 무엇인가요?",
    "외국인 근로자가 불법적으로 근로 조건이 변경되었을 때 이를 막을 수 있는 법률은 무엇인가요?",
    "외국인 근로자가 업무 중 질병에 걸렸을 때 보호받을 수 있는 법률 목록을 알려주세요.",
    "외국인 근로자가 고용 계약 종료 후 체불 임금을 받을 수 있는 법적 권리와 관련된 법률은 무엇인가요?",
    "외국인 근로자가 한국에서 노동권을 침해당했을 때 구제받을 수 있는 법률 목록을 알고 싶습니다.",
    "외국인 근로자가 노동조합을 설립하거나 참여할 때 적용될 수 있는 법률에는 무엇이 있나요?",
    
    # 절차 및 제도 관련 질문 (5~10개)
    "외국인 근로자가 임금을 체불당했을 때 신고할 수 있는 절차와 관련된 법률은 무엇인가요?",
    "외국인 근로자가 부당 해고를 당했을 때 법적으로 취할 수 있는 절차는 무엇인가요?",
    "외국인 근로자가 산업재해를 당했을 때 보상을 받기 위한 절차와 관련된 법률 목록을 알려주세요.",
    "외국인 근로자가 최저임금 이하로 받았을 때 법적으로 신고할 수 있는 절차는 무엇인가요?",
    "한국에서 외국인 근로자가 법적으로 취업 비자를 갱신하는 절차는 무엇인가요?",
    "외국인 근로자가 불법적으로 해고당했을 때 구제를 받기 위한 절차는 무엇인가요?",
    "외국인 근로자가 근로 환경 개선을 요구할 수 있는 법적 절차는 무엇인가요?",
    "외국인 근로자가 고용주로부터 받은 불이익을 신고할 수 있는 법적 절차는 무엇인가요?",
    "외국인 근로자가 임금을 받지 못했을 때 법적 구제 절차를 통해 받을 수 있는 혜택은 무엇인가요?",
    "외국인 근로자가 산업재해 보상 보험에 가입할 수 있는 절차와 관련된 법률은 무엇인가요?"
]

In [4]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [18]:
import openpyxl
import time


workbook = openpyxl.Workbook()

sheet = workbook.active
sheet.title = "RAG_test_results"
sheet.append(["질문", "답변", "참고 문서", "응답 시간(s)"])

# 각 질문에 대해 응답 및 응답 시간 확인
for question in questions:
    start = time.time()
    answer, source_docs = ask_labor_law_question(rag_chain, question)
    response_time = str(round(time.time() - start,2))+"s"
    print(response_time)
    src =""
    if source_docs:
        for idx, doc in enumerate(source_docs, 1):
            src +=str(idx)+'. '+doc.metadata['law'] +" "+ doc.metadata['chapter'] +" "+ doc.metadata['title'] + "\n"
    sheet.append([question, answer, src, response_time])

workbook.save("Labor Law Chatbot Results.xlsx")

18.37s


## 5. CoVe

1. Generation Baseline Response : RAG 기반으로 LLM 을 통해서 일반적인 초기 답변을 받음
2. Plan Verifications : few-shot prompt 를 구성해서 빠르게 검증 질문 생성
3. Execute Verifications : RAG 를 통해서 plan verification 에 대한 답변 생성
4. Generate Final Verified Response : Based line , 검증 질문, 검증 답변을 Domain knowledge 로 사용해서 few-shot 을 통해서 개선된 답변 도출.

### 1. Generation Baseline Response

In [10]:
# Baseline Response 생성 (RAG를 사용하여 초기 답변 생성)
initial_question = "외국인 근로자가 근로 환경 개선을 요구할 수 있는 법적 절차는 무엇인가요?"
baseline_answer, baseline_docs = ask_labor_law_question(rag_chain, initial_question)

print(f"[답변]: {baseline_answer}\n")
print("[참고 문서]:")
if baseline_docs:
    for idx, doc in enumerate(baseline_docs, 1):
        print(f"{idx}. {doc.metadata['law']}, {doc.metadata['chapter']}, {doc.metadata['title']}")
else:
    print("참조된 문서가 없습니다.")

[답변]: 외국인 근로자가 근로 환경 개선을 요구할 수 있는 법적 절차는 여러 가지가 있습니다. 주로 관련 법률에 따라 진행되며, 다음과 같은 절차를 따를 수 있습니다.

1. **고충처리위원회 활용**: 모든 사업장에는 근로자의 고충을 청취하고 이를 처리하기 위한 고충처리위원회를 두어야 합니다. 다만, 상시 30명 미만의 근로자를 사용하는 사업장은 제외됩니다. 외국인 근로자는 이 위원회를 통해 자신의 고충을 제기할 수 있습니다. 이 조항은 "근로자참여 및 협력증진에 관한 법률" 제25조에 명시되어 있습니다.

2. **근로계약서 검토**: 외국인 근로자는 근로계약서에 명시된 근로 조건을 검토하고, 만약 계약서에 명시된 조건이 지켜지지 않는 경우, 이를 근거로 고용주에게 개선을 요구할 수 있습니다. 근로계약서는 "근로기준법"에 따라 작성되어야 하며, 근로자의 권리를 보호하는 중요한 문서입니다.

3. **노동청에 신고**: 근로 환경 개선이 이루어지지 않거나 고용주가 요구를 무시할 경우, 외국인 근로자는 관할 노동청에 신고할 수 있습니다. 노동청은 근로자의 권리를 보호하고, 고용주에게 법적 조치를 취할 수 있는 권한을 가지고 있습니다.

4. **법적 조치**: 만약 고충처리위원회나 노동청을 통한 해결이 불가능할 경우, 외국인 근로자는 법원에 소송을 제기할 수 있습니다. 이 경우, 법률 전문가의 도움을 받는 것이 좋습니다.

이러한 절차를 통해 외국인 근로자는 자신의 근로 환경을 개선하기 위한 법적 권리를 행사할 수 있습니다. 만약 구체적인 상황에 대한 조언이 필요하다면, 전문 노동 변호사와 상담하는 것이 좋습니다.

[참고 문서]:
1. 고용보험법, 제3장 고용안정ㆍ직업능력개발 사업, 제20조(고용창출의 지원)
2. 근로자참여 및 협력증진에 관한 법률, 제5장 고충처리, 제26조(고충처리위원)
3. 외국인근로자의 고용 등에 관한 법률, 제2장 외국인근로자 고용절차, 제6조(내국인 구인 노력)
4. 외국인근로자의 고용 등에 관한 법률, 제2장 외국인근로자 고용

### 2. Plan Verification

In [11]:
# 검증 질문을 여러 개 생성하는 프롬프트 예시
verification_prompt = """
Please create additional verification questions for the existing answers.
Original answer: {baseline_answer}

Verification questions:
1. ...
2. ...
3. ...
"""

# LLM을 통해 검증 질문 생성
verification_prompt_formatted = verification_prompt.format(baseline_answer=baseline_answer)

# 메시지 형식으로 전달
messages = [
    {"role": "system", "content": "You are an assistant that helps generate verification questions."},
    {"role": "user", "content": verification_prompt_formatted}
]

# LLM 호출
verification_questions = llm(messages=messages)

# 검증 질문이 여러 개 생성된다고 가정 (각각의 질문을 리스트로 받음)
verification_questions_list = verification_questions.content.split("\n")
verification_questions_list = [q for q in verification_questions_list if q.strip()]  # 빈 줄 제거

print('Verification questions:', verification_questions_list)[1:]

Verification questions: ['Verification questions:', '1. 외국인 근로자가 고충처리위원회를 통해 제기할 수 있는 고충의 종류는 무엇인가요?', '2. "근로자참여 및 협력증진에 관한 법률" 제25조에 명시된 고충처리위원회의 역할은 무엇인가요?', '3. 근로계약서에 명시된 조건이 지켜지지 않을 경우, 외국인 근로자는 어떤 절차를 통해 개선을 요구할 수 있나요?', '4. 노동청에 신고하기 전에 외국인 근로자가 고려해야 할 사항은 무엇인가요?', '5. 법적 조치를 취하기 위해 외국인 근로자가 준비해야 할 서류나 증거는 어떤 것이 있나요?', '6. 외국인 근로자가 법원에 소송을 제기할 경우, 어떤 법률 전문가의 도움을 받는 것이 좋나요?', '7. 외국인 근로자가 근로 환경 개선을 요구할 때, 어떤 법률이 그들의 권리를 보호하나요?', '8. 상시 30명 미만의 근로자를 사용하는 사업장에서 외국인 근로자는 어떤 절차를 통해 고충을 제기할 수 있나요?']


In [12]:
verification_questions_list[1:]

['1. 외국인 근로자가 고충처리위원회를 통해 제기할 수 있는 고충의 종류는 무엇인가요?',
 '2. "근로자참여 및 협력증진에 관한 법률" 제25조에 명시된 고충처리위원회의 역할은 무엇인가요?',
 '3. 근로계약서에 명시된 조건이 지켜지지 않을 경우, 외국인 근로자는 어떤 절차를 통해 개선을 요구할 수 있나요?',
 '4. 노동청에 신고하기 전에 외국인 근로자가 고려해야 할 사항은 무엇인가요?',
 '5. 법적 조치를 취하기 위해 외국인 근로자가 준비해야 할 서류나 증거는 어떤 것이 있나요?',
 '6. 외국인 근로자가 법원에 소송을 제기할 경우, 어떤 법률 전문가의 도움을 받는 것이 좋나요?',
 '7. 외국인 근로자가 근로 환경 개선을 요구할 때, 어떤 법률이 그들의 권리를 보호하나요?',
 '8. 상시 30명 미만의 근로자를 사용하는 사업장에서 외국인 근로자는 어떤 절차를 통해 고충을 제기할 수 있나요?']

### 3. Execute Verification

In [22]:
# 각 검증 질문에 대해 RAG 기반 검증 답변 생성
verification_answers = []
verification_docs_list = []

for i, verification_question in enumerate(verification_questions_list):
    verification_answer, verification_docs = ask_labor_law_question(rag_chain, verification_question)
    print("=====")
    print(i)
    print("question : ", verification_question)
    print("answer : ", verification_answer)
    print("=====")
    verification_answers.append(verification_answer)
    verification_docs_list.append(verification_docs)

=====
0
question :  Verification questions:
answer :  I'm sorry, but I don't have enough information to answer your question about verification. If you could provide more context or specify what you need to verify, I would be happy to help!
=====
=====
1
question :  1. 외국인 근로자가 고충처리위원회를 통해 제기할 수 있는 고충의 종류는 무엇인가요?
answer :  외국인 근로자가 고충처리위원회를 통해 제기할 수 있는 고충의 종류는 법률에서 구체적으로 명시되어 있지 않지만, 일반적으로 고충처리위원회는 근로자와 관련된 다양한 문제를 다룰 수 있습니다. 이러한 문제는 다음과 같은 범주에 포함될 수 있습니다:

1. **근로 조건**: 임금, 근무 시간, 휴가, 근로 환경 등과 관련된 문제.
2. **차별 및 괴롭힘**: 인종, 성별, 국적 등에 따른 차별이나 직장 내 괴롭힘.
3. **계약 문제**: 근로 계약의 내용이나 해지와 관련된 문제.
4. **안전 및 건강**: 작업 환경의 안전성이나 건강 문제.
5. **기타**: 근로자의 권리와 관련된 기타 문제.

고충처리위원회는 근로자로부터 고충사항을 청취한 후, 10일 이내에 조치 사항과 처리 결과를 해당 근로자에게 통보해야 합니다(근로자참여 및 협력증진에 관한 법률 제25조). 

이와 관련된 법률은 "근로자참여 및 협력증진에 관한 법률"이며, 고충처리위원회의 설치 및 운영에 대한 규정은 제25조와 제26조에서 다루고 있습니다. 

고충처리위원회에 대한 더 자세한 정보는 고용노동부의 관련 부서에서 확인할 수 있습니다.
=====
=====
2
question :  2. "근로자참여 및 협력증진에 관한 법률" 제25조에 따르면, 고충처리위원회를 두어야 하는 사업장의 최소 근로자 수는 몇 명인가요?
ans

### 4. Final Answer

In [25]:
# 개선된 답변을 도출하는 프롬프트 생성
improvement_prompt = """
"You are an AI chatbot specialized in Korean labor law. "
"Please explain in as much detail as possible."
"Please mention exactly which 'law' and 'title' you referenced in the retrieved context."
"If possible, please also provide the number or link of the relevant department."
"Please explain it to me as easily as possible, like a friend."

Please write a final improved answer based on the initial question, existing answer, verification question, and verification answer.

Initial question: {initial_question}
Original answer: {baseline_answer}
Verification question and answer:
{verification_q_and_a}

Final improved answer:
"""

# 검증 질문과 그에 대한 답변을 결합
verification_q_and_a = "\n".join(
    [f"검증 질문: {q}\n검증 답변: {a}" for q, a in zip(verification_questions_list, verification_answers)]
)

# 메시지 형식으로 LLM에 전달할 내용 구성
messages = [
    {"role": "system", "content": "You are an AI chatbot specialized in Korean labor law."},
    {"role": "user", "content": improvement_prompt.format(
        initial_question=initial_question,
        baseline_answer=baseline_answer,
        verification_q_and_a=verification_q_and_a
    )}
]

# LLM 호출
improved_answer = llm(messages=messages)

# 최종 개선된 답변 출력
print("최종 개선된 답변:")
print(improved_answer.content)

최종 개선된 답변:
외국인 근로자가 근로 환경 개선을 요구할 수 있는 법적 절차는 여러 가지가 있으며, 주로 관련 법률에 따라 진행됩니다. 아래에서 자세히 설명드리겠습니다.

### 1. 고충처리위원회 활용
모든 사업장에는 근로자의 고충을 청취하고 처리하기 위한 고충처리위원회를 두어야 합니다. 다만, 상시 30명 미만의 근로자를 사용하는 사업장은 예외입니다. 이 조항은 **"근로자참여 및 협력증진에 관한 법률" 제25조**에 명시되어 있습니다. 외국인 근로자는 이 위원회를 통해 자신의 고충을 제기할 수 있습니다.

### 2. 근로계약서 검토
외국인 근로자는 자신의 근로계약서를 검토하여 근로 조건이 법적으로 정해진 기준에 부합하는지 확인할 수 있습니다. 만약 근로 조건이 불합리하다면, 이를 근거로 개선을 요구할 수 있습니다. 주요 확인 요소는 근로시간, 임금, 해고 조건 등입니다.

### 3. 노동청 신고
근로 환경이 법적으로 보장된 기준에 미치지 못할 경우, 외국인 근로자는 관할 노동청에 신고할 수 있습니다. 노동청은 근로자의 권리를 보호하기 위해 필요한 조치를 취할 수 있습니다. 이와 관련된 법률은 **"근로기준법"**입니다.

### 4. 법률 상담
필요시, 외국인 근로자는 법률 상담을 통해 자신의 권리를 보호할 수 있는 방법에 대해 조언을 받을 수 있습니다. 이는 전문 변호사나 노동 관련 단체를 통해 이루어질 수 있습니다. 법률 상담을 받을 때는 구체적인 상황을 설명하고 필요한 서류를 준비하는 것이 좋습니다.

### 준비해야 할 서류 및 증거
근로 환경 개선을 요구하기 위해 준비해야 할 서류나 증거는 다음과 같습니다:
- **근로계약서**: 근로 조건과 환경에 대한 기본적인 내용이 포함되어야 합니다.
- **근로 환경 관련 사진 또는 비디오**: 개선이 필요한 부분을 명확히 하는 데 유용합니다.
- **동료 근로자들의 진술서**: 문제의 심각성을 강조하는 데 도움이 됩니다.
- **의료 기록**: 근로 환경이 건강에 미치는 영향을 증명할 수 있는 자

### RAG + CoVe TEST

In [4]:
import openpyxl
import time

# save to excel
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "RAG_CoVe_test_results"
sheet.append(["index", "initial question", "baseline answer", "documents", "RAG response time(s)", "verification questions", "verification answers", "final answer", 'total response time(s)'])

for question in questions:
    # Baseline Response 생성 (RAG를 사용하여 초기 답변 생성)
    initial_question = question
    start = time.time()
    baseline_answer, baseline_docs = ask_labor_law_question(rag_chain, initial_question)
    print("Generate baseline Response")
    rag_end = time.time()
    
    baseline_doc_string = ""
    if baseline_docs:
        for idx, doc in enumerate(baseline_docs, 1):
            baseline_doc_string += f"{idx}. {doc.metadata['law']} {doc.metadata['chapter']} {doc.metadata['title']}\n"
    else:
        baseline_doc_string += "참조된 문서가 없습니다.\n"
    
    # 검증 질문을 여러 개 생성하는 프롬프트
    verification_prompt = """
    Please create additional verification questions for the existing answers.
    Original answer: {baseline_answer}
    
    Verification questions:
    1. ...
    2. ...
    3. ...
    """
    
    # LLM을 통해 검증 질문 생성
    verification_prompt_formatted = verification_prompt.format(baseline_answer=baseline_answer)
    
    # 메시지 형식으로 전달
    messages = [
        {"role": "system", "content": "You are an assistant that helps generate verification questions."},
        {"role": "user", "content": verification_prompt_formatted}
    ]
    
    # LLM 호출
    verification_questions = llm(messages=messages)
    print("Generate verification question")
    
    verification_questions_list = verification_questions.content.split("\n")
    verification_questions_list = [q for q in verification_questions_list if q.strip()]  # 빈 줄 제거
    verification_questions_list = verification_questions_list[1:]   # 제목 제거
    
    # 각 검증 질문에 대해 RAG 기반 검증 답변 생성
    verification_answers = []
    verification_docs_list = []
    
    for i, verification_question in enumerate(verification_questions_list):
        verification_answer, verification_docs = ask_labor_law_question(rag_chain, verification_question)
        print(f"{i+1}. verification answer")
    
        verification_answers.append(verification_answer)
        verification_docs_list.append(verification_docs)
    
    # 개선된 답변을 도출하는 프롬프트 생성
    improvement_prompt = """
    "You are an AI chatbot specialized in Korean labor law. "
    "Please explain in as much detail as possible."
    "Please mention exactly which 'law' and 'title' you referenced in the retrieved context."
    "If possible, please also provide the number or link of the relevant department."
    "Please explain it to me as easily as possible, like a friend."
    
    Please write a final improved answer based on the initial question, existing answer, verification question, and verification answer.
    
    Initial question: {initial_question}
    Original answer: {baseline_answer}
    Verification question and answer:
    {verification_q_and_a}
    
    Final improved answer:
    """
    
    # 검증 질문과 그에 대한 답변을 결합
    verification_q_and_a = "\n".join(
        [f"검증 질문: {q}\n검증 답변: {a}" for q, a in zip(verification_questions_list, verification_answers)]
    )
    
    # 메시지 형식으로 LLM에 전달할 내용 구성
    messages = [
        {"role": "system", "content": "You are an AI chatbot specialized in Korean labor law."},
        {"role": "user", "content": improvement_prompt.format(
            initial_question=initial_question,
            baseline_answer=baseline_answer,
            verification_q_and_a=verification_q_and_a
        )}
    ]
    
    # LLM 호출
    improved_answer = llm(messages=messages)
    cove_end = time.time()
    rag_time = round(rag_end - start, 2)
    cove_time = round(cove_end - start, 2)
    
    # baseline 기록
    sheet.append([1, initial_question, baseline_answer, baseline_doc_string, rag_time, "", "", improved_answer.content, cove_time])
    
    # 각 검증 질문과 답변을 기록
    for i, (verify_q, verify_a) in enumerate(zip(verification_questions_list, verification_answers), 2):
        sheet.append(["", "", "", "", "", verify_q, verify_a, "", ""])

# Excel 파일 저장
workbook.save("RAG_CoVe_test_results.xlsx")
print("save RAG_CoVe_test_results.xlsx")

Generate baseline Response


/var/folders/6v/trwfv86n72l0_209w0djx9g00000gn/T/ipykernel_83523/2663782029.py:46: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  verification_questions = llm(messages=messages)


Generate verification question
1. verification answer
2. verification answer
3. verification answer
4. verification answer
5. verification answer
6. verification answer
7. verification answer
8. verification answer
9. verification answer
10. verification answer
Generate baseline Response
Generate verification question
1. verification answer
2. verification answer
3. verification answer
4. verification answer
5. verification answer
6. verification answer
7. verification answer
8. verification answer
Generate baseline Response
Generate verification question
1. verification answer
2. verification answer
3. verification answer
4. verification answer
5. verification answer
6. verification answer
7. verification answer
8. verification answer
Generate baseline Response
Generate verification question
1. verification answer
2. verification answer
3. verification answer
4. verification answer
5. verification answer
6. verification answer
7. verification answer
Generate baseline Response
Generate

## 6. Conversational RAG
- https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/